In [1]:
%load_ext autoreload
%autoreload 2

from glob import glob
import os
import pandas as pd

from mocca.campaign.model import HplcDadCampaign
from mocca.campaign.process_funcs import process_compound_experiments
from mocca.campaign.settings import Settings
from mocca.campaign.experiment import Experiment
from mocca.campaign.compound import Compound
from mocca.campaign.istd import InternalStandard

from mocca.dad_data.models import CompoundData, GradientData

In [2]:
# get the folders matching flag in directory
flag = '.txt'

###
# Add the path to the provided data folder
test_data_path = '/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data' # directory of test data
###

folders = glob(test_data_path + '/*' + flag + '*') #find all matching flag
folders = sorted(folders, key=lambda x: int(x.split('_')[-1][:-4]))

In [3]:
istd_folder = [folder for folder in folders if folder.split('_')[-1][:-4] == '96']

product_indices =['92', '93', '94', '95']
product_folders = [folder for folder in folders if folder.split('_')[-1][:-4] in product_indices]

substrate_indices =['88', '89', '90', '91']
substrate_folders = [folder for folder in folders if folder.split('_')[-1][:-4] in substrate_indices]

# obviously, we used a blank injection as the gradient run
gradient_folder = [folder for folder in folders if folder.split('_')[-1][:-4] == '97']

In [4]:
# remove some data which was gathered due to repeated start of the campaigns
bad_indices =['0', '1', '2', '3']
bad_folders = [folder for folder in folders if folder.split('_')[-1][:-4] in bad_indices]

cnsource_indices =['98', '99']
cnsource_folders = [folder for folder in folders if folder.split('_')[-1][:-4] in cnsource_indices]

remove_folders = bad_folders + istd_folder + product_folders + substrate_folders + gradient_folder + cnsource_folders

reaction_folders = [folder for folder in folders if folder not in remove_folders]

In [5]:
gradient_experiment = Experiment(gradient_folder[0], gradient=True)

In [6]:
campaign = HplcDadCampaign('labsolutions')

In [7]:
campaign.settings

Settings(hplc_system_tag='labsolutions', detector_limit=2000, absorbance_threshold=None, wl_high_pass=None, wl_low_pass=None, peaks_high_pass=None, peaks_low_pass=None, spectrum_correl_thresh=None, relative_distance_thresh=None)

In [8]:
ArCl_concs = [0.0603, 0.04422, 0.03015, 0.01407]
ArCN_concs = [0.05955, 0.04367, 0.029775, 0.013895]
istd_conc = 0.06094

In [9]:
istd_experiment = Experiment(istd_folder[0], compound=Compound('tetralin', istd_conc, istd=True))

In [10]:
substrate_experiments = []
for i, conc in enumerate(ArCl_concs):
    exp = Experiment(substrate_folders[i], compound=Compound('2-chlorotuluene', conc), istd=InternalStandard('tetralin', istd_conc))
    substrate_experiments.append(exp)

In [11]:
product_experiments = []
for i, conc in enumerate(ArCN_concs):
    exp = Experiment(product_folders[i], compound=Compound('o-tolunitrile', conc), istd=InternalStandard('tetralin', istd_conc))
    product_experiments.append(exp)

In [12]:
campaign.add_experiment(gradient_experiment)
campaign.add_experiment(istd_experiment)
for exp in product_experiments:
    campaign.add_experiment(exp)
for exp in substrate_experiments:
    campaign.add_experiment(exp)
campaign.add_experiment(Experiment(bad_folders[0], compound=Compound("a")))

In [13]:
campaign.experiments

[Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_97.txt', compound=None, istd=None, gradient=True, processed=False),
 Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_96.txt', compound=Compound(key='tetralin', conc=0.06094, solvent=False, istd=True), istd=None, gradient=False, processed=False),
 Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_92.txt', compound=Compound(key='o-tolunitrile', conc=0.05955, solvent=False, istd=False), istd=[InternalStandard(key='tetralin', conc=0.06094)], gradient=False, processed=False),
 Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_93.txt', compound=Compound(key='o-tolunitrile', conc=0.04367, solvent=False, istd=False)

In [17]:
compound_chroms, bad_chroms = campaign.process_all_experiments(detector_limit=campaign.settings.detector_limit, wl_low_pass=None,
                                 spectrum_correl_thresh=0.95, relative_distance_thresh=0.01,
                                 wl_high_pass = 215, peaks_high_pass = 1, 
                                 peaks_low_pass = 5, absorbance_threshold = 700)

/Users/haascp/opt/anaconda3/envs/mocca-dev/lib/python3.9/site-packages/sklearn/metrics/_regression.py:781: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [21]:
for chrom in compound_chroms:
    display(chrom.dataset.path)
    for peak in chrom.peaks:
        display(peak.compound_id)
        display(peak.integral)
        #display(peak.concentration)
        display(peak.istd)

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_96.txt'

'2-chlorotuluene'

10618.060848096951

[]

'tetralin'

...

101291.93802706029

[IstdPeak(maximum=1405, integral=39231.976970444244, offset=-1, compound_id='tetralin', concentration=0.06094)]

'tetralin'

39231.976970444244

[IstdPeak(maximum=1405, integral=39231.976970444244, offset=-1, compound_id='tetralin', concentration=0.06094)]

In [18]:
for comp in campaign.quali_comp_db.items:
    display(comp.compound_id)
    display(len(comp.created_from))


'tetralin'

6

'2-chlorotuluene'

3

'o-tolunitrile'

3

In [19]:
for comp in campaign.quant_comp_db.items:
    display(comp.compound_id)
    display(comp.calib_factors)
    display(comp.calib_data)    
    display(len(comp.created_from))

'tetralin'

{'absolute': 1489420.15472096}

{'absolute': [(0.06094, 90765.2642286953)]}

1

'2-chlorotuluene'

...

2

'o-tolunitrile'

{'absolute': 3237864.687574665, 'tetralin': 5.318045099264461}

{'absolute': [(0.05955, 203579.565272476),
  (0.04367, 123388.0305363853),
  (0.029775, 101291.93802706029)],
 'tetralin': [(0.05955, 0.3178851955827124),
  (0.04367, 0.23129423456953574),
  (0.029775, 0.15733927219674235)]}

3

In [141]:
for chrom in bad_chroms:
    display(chrom.dataset.path)
    for peak in chrom.peaks:
        display(peak.compound_id)
        display(peak.integral)

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_90.txt'

'tetralin'

16856.97919906799

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_91.txt'

'tetralin'

32323.18913020689

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_95.txt'

'o-tolunitrile'

16509.21486814634

AttributeError: 'NoneType' object has no attribute 'path'

In [114]:
display(bad_chroms[0].dataset)

None

In [96]:
compound_id = '2-chlorotuluene_impurity_1'

In [97]:
'impurity' in compound_id

True

In [32]:
campaign.gradient

In [18]:
test = CompoundData('labsolutions', istd_experiment, GradientData('labsolutions', gradient_experiment, wl_high_pass = 250), wl_high_pass = 250)

In [19]:
test

CompoundData(hplc_system_tag='labsolutions', experiment=Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_96.txt', compound=Compound(key='tetralin', conc=0.06094, solvent=False, istd=True), istd=None, gradient=False, processed=True), path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_96.txt', data=array([[0.05222568, 0.05281713, 0.05320858, ..., 0.02930742, 0.03026063,
        0.03101384],
       [0.04542254, 0.04611323, 0.04600392, ..., 0.03151315, 0.03181611,
        0.03191907],
       [0.05019863, 0.05081775, 0.05243686, ..., 0.03352349, 0.03481353,
        0.03610356],
       ...,
       [0.00985935, 0.00982639, 0.00959342, ..., 0.00976906, 0.00859204,
        0.00661501],
       [0.00773878, 0.00772346, 0.00830815, ..., 0.00939489, 0.00921391,
        0.00863293],
       [0.        , 0.        , 0.        , ..., 0.        , 0.  

In [146]:
test = ((1, 2), (2, 4), (3, 6), (4, 8))

In [148]:
for i in zip(*test):
    print(i)

(1, 2, 3, 4)
(2, 4, 6, 8)
